## RNN Implementation

### Data and imports   

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf

#### Get Data from Yahoo Finance

In [4]:
dfs = {}
yfinance_tickers = ['^GSPC'] # Ticker for the SP500, which is arguably the most influential, we start with just this

# Got this code from https://www.kaggle.com/code/xxxxyyyy80008/predict-stock-market-crashes 
for ticker in  yfinance_tickers:
    cur_data = yf.Ticker(ticker)
    hist = cur_data.history(period="max")
    print(ticker, cur_data.info['shortName'], datetime.now(), ticker, hist.shape, hist.index.min(), hist.index.max())

    dfs[ticker] = hist

^GSPC S&P 500 2024-11-28 21:03:36.531791 ^GSPC (24345, 7) 1927-12-30 00:00:00-05:00 2024-11-27 00:00:00-05:00


## ALL CODE (Easier to work with)

### DATA WORK

In [ ]:
## ALL CODE

import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf


dfs = {} # Store all the data for our tickers, even though for now its just SP500
tickers = ['^GSPC',] # Ticker (identifier) for the SP500, which is arguably the most influential, we start with just this

# Got this code from https://www.kaggle.com/code/xxxxyyyy80008/predict-stock-market-crashes 
for ticker in tickers:
    cur_data = yf.Ticker(ticker)
    hist = cur_data.history(period="max")
    print(ticker, cur_data.info['shortName'], datetime.now(), ticker, hist.shape, hist.index.min(), hist.index.max())
    dfs[ticker] = hist


'''
1. Calculate the drawdown and assign days with drawdown in the 99.5% quantile as crash (i.e. label=1). 
2. Calculate the 15-day percent change, and assign days that with value within 5% quantile as crash. 
   The target is shift 3 days to make the problem as "Predict if there will be a market crash 3 days later".
'''



# Calculates how much the price has gone down from its highest point, labeling those with a 0.005 quantile as crashes
# got help from chatgpt for this function
def calculate_drawdown_and_label(data, quantile_threshold=0.005):

    data['daily_return'] = data['Close'].pct_change()

    data['drawdown'] = data['Close'] / data['Close'].cummax() - 1

    crash_threshold = data['drawdown'].quantile(quantile_threshold)

    data['crash_label'] = (data['drawdown'] < crash_threshold).astype(int)

    return data

## Make the data we targetted the target, we try to predict the market crashes 3 days in advance so shift_days = 3
def prepare_target(data, shift_days=3):
    data['target'] = (data['crash_label']).astype(int).shift(-shift_days)
    return data

# prep the data
for ticker, data in dfs.items():
    data = calculate_drawdown_and_label(data)
    data = prepare_target(data)
    dfs[ticker] = data


# now we have a database and targetted data, we need to train our RNN on it
### RNN



^GSPC S&P 500 2024-11-28 21:57:11.460396 ^GSPC (24345, 7) 1927-12-30 00:00:00-05:00 2024-11-27 00:00:00-05:00


### RNN